# Graph-Enhanced Vector Retriever

To take advantage of the relationships in the graph, you can create a retriever that uses both vector search and graph traversal to find relevant data.

The `VectorCypherRetriever` allows you to perform vector searches and then traverse the graph to find related nodes or entities.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from neo4j import GraphDatabase
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever

# Connect to Neo4j database
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USERNAME"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

# Create embedder
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")


## Retrieval Query

The retrieval query is a Cypher query that will be used to get data from the graph after the nodes are returned by the vector search.

The query receives the node and score variables yielded by the vector search.

Add this retrieval query to the code:

In [2]:
# Define retrieval query
retrieval_query = """
MATCH (node)<-[r:RATED]-()
RETURN 
  node.title AS title, node.plot AS plot, score AS similarityScore, 
  collect { MATCH (node)-[:IN_GENRE]->(g) RETURN g.name } as genres, 
  collect { MATCH (node)<-[:ACTED_IN]->(a) RETURN a.name } as actors, 
  avg(r.rating) as userRating
ORDER BY userRating DESC
"""

## Retriever

You can now use the `VectorCypherRetriever` class to create a retriever that will perform the vector search and then traverse the graph:

In [3]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

# Create retriever
retriever = VectorCypherRetriever(
    driver,
    index_name="moviePlots",
    embedder=embedder,
    retrieval_query=retrieval_query,
)

In [4]:
from neo4j_graphrag.llm import OpenAILLM

#  Create the LLM
llm = OpenAILLM(model_name="gpt-4o")

In [5]:
from neo4j_graphrag.generation import GraphRAG

# Create GraphRAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

In [8]:
# Search
query_text = "Find the highest rated action movie about travelling to other planets"

response = rag.search(
    query_text=query_text, 
    retriever_config={"top_k": 10},
    return_context=True
)

print(response.answer)
print("CONTEXT:", response.retriever_result.items)

The highest-rated action movie about traveling to other planets in the provided context is "Screamers" with a user rating of 3.333333333333333.
CONTEXT: [RetrieverResultItem(content="<Record title='City of Lost Children, The (Cité des enfants perdus, La)' plot='A scientist in a surrealist society kidnaps children to steal their dreams, hoping that they slow his aging process.' similarityScore=0.8912997245788574 genres=['Adventure', 'Mystery', 'Fantasy', 'Drama', 'Sci-Fi'] actors=['Judith Vittet', 'Daniel Emilfork', 'Dominique Pinon', 'Ron Perlman'] userRating=4.025>", metadata=None), RetrieverResultItem(content="<Record title='Twelve Monkeys (a.k.a. 12 Monkeys)' plot='In a future world devastated by disease, a convict is sent back in time to gather information about the man-made virus that wiped out most of the human population on the planet.' similarityScore=0.8899544477462769 genres=['Thriller', 'Sci-Fi', 'Mystery'] actors=['Michael Chance', 'Joseph Melito', 'Bruce Willis', 'Jon Seda

In [14]:
retrieval_query = """
MATCH (node)<-[:DIRECTED]-(director)
RETURN node.title, director.name
"""

retriever = VectorCypherRetriever(
    driver,
    index_name="moviePlots",
    embedder=embedder,
    retrieval_query=retrieval_query,
)

rag = GraphRAG(retriever=retriever, llm=llm)

query_text = "Who has directed movies about weddings?"

response = rag.search(
    query_text=query_text, 
    retriever_config={"top_k": 10},
    return_context=True
)

print(response.answer)
print("CONTEXT:", response.retriever_result.items)

Charles Shyer directed "Father of the Bride Part II," which involves a wedding theme.
CONTEXT: [RetrieverResultItem(content="<Record node.title='Father of the Bride Part II' director.name='Charles Shyer'>", metadata=None), RetrieverResultItem(content="<Record node.title='How to Make an American Quilt' director.name='Jocelyn Moorhouse'>", metadata=None), RetrieverResultItem(content="<Record node.title='Bed of Roses' director.name='Michael Goldenberg'>", metadata=None), RetrieverResultItem(content="<Record node.title='American President, The' director.name='Rob Reiner'>", metadata=None), RetrieverResultItem(content="<Record node.title='Get Shorty' director.name='Barry Sonnenfeld'>", metadata=None), RetrieverResultItem(content="<Record node.title='To Die For' director.name='Gus Van Sant'>", metadata=None), RetrieverResultItem(content="<Record node.title='French Twist (Gazon maudit)' director.name='Josiane Balasko'>", metadata=None), RetrieverResultItem(content="<Record node.title='When Ni

Complete code

```python
import os
from dotenv import load_dotenv
load_dotenv()

from neo4j import GraphDatabase
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorCypherRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG

# Connect to Neo4j database
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USERNAME"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

# Create embedder
embedder = OpenAIEmbeddings(model="text-embedding-ada-002")

# Define retrieval query
retrieval_query = """
MATCH (node)<-[r:RATED]-()
RETURN 
  node.title AS title, node.plot AS plot, score AS similarityScore, 
  collect { MATCH (node)-[:IN_GENRE]->(g) RETURN g.name } as genres, 
  collect { MATCH (node)<-[:ACTED_IN]->(a) RETURN a.name } as actors, 
  avg(r.rating) as userRating
ORDER BY userRating DESC
"""

# Create retriever
retriever = VectorCypherRetriever(
    driver,
    index_name="moviePlots",
    embedder=embedder,
    retrieval_query=retrieval_query,
)

#  Create the LLM
llm = OpenAILLM(model_name="gpt-4o")

# Create GraphRAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

# Search
query_text = "Find the highest rated action movie about travelling to other planets"

response = rag.search(
    query_text=query_text, 
    retriever_config={"top_k": 5},
    return_context=True
)

print(response.answer)
print("CONTEXT:", response.retriever_result.items)

# Close the database connection
driver.close()
```